<a href="https://colab.research.google.com/github/bpratham2001/Servilia/blob/main/notebooks/Wine_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
#!pip install kagglehub[pandas-datasets]
!pip install opendatasets

In [42]:
import opendatasets as od
import pandas as pd
import numpy as np

#checked
#mysarahmadbhat/wine-tasting // winemag-data-130k-v2.csv \\ zynicide/wine-reviews is better

#potential sources
# https://en.wikipedia.org/wiki/Wine_tasting_descriptors
# https://www.wineenthusiast.com/toplists/wine/
# https://www.wine-tastings-guide.com/wine-descriptions.html
# https://en.wikipedia.org/wiki/Wine_tasting

od.download("https://www.kaggle.com/datasets/zynicide/wine-reviews", force=True) # winemag-data_first150k.csv, winemag-data-130k-v2.csv
file =('wine-reviews/winemag-data_first150k.csv')
df1 = pd.read_csv(file)
file =('wine-reviews/winemag-data-130k-v2.csv')
df2 = pd.read_csv(file)

od.download("https://www.kaggle.com/datasets/fedesoriano/spanish-wine-quality-dataset", force=True) # wines_SPA.csv
file =('spanish-wine-quality-dataset/wines_SPA.csv')
df3 = pd.read_csv(file)

od.download("https://www.kaggle.com/datasets/elvinrustam/wine-dataset", force=True) # WineDataset.csv
file =('wine-dataset/WineDataset.csv')
df4 = pd.read_csv(file)

Dataset URL: https://www.kaggle.com/datasets/zynicide/wine-reviews


100%|██████████| 50.9M/50.9M [00:00<00:00, 190MB/s]



Dataset URL: https://www.kaggle.com/datasets/fedesoriano/spanish-wine-quality-dataset


100%|██████████| 46.4k/46.4k [00:00<00:00, 25.3MB/s]


Dataset URL: https://www.kaggle.com/datasets/elvinrustam/wine-dataset


100%|██████████| 308k/308k [00:00<00:00, 73.6MB/s]

In [37]:
df1.rename(columns={'Unnamed: 0': 'id'}, inplace=True)
df2.rename(columns={'Unnamed: 0': 'id'}, inplace=True)
#df['DataFrame Column'] = df['DataFrame Column'].fillna(0)

for i in df1.keys():
  print(i +" "+ str(df1[i].isnull().values.any()))
df1.isnull().values.any()
print("\n")
for i in df2.keys():
  print(i +" "+ str(df2[i].isnull().values.any()))

id False
country True
description False
designation True
points False
price True
province True
region_1 True
region_2 True
variety False
winery False


id False
country True
description False
designation True
points False
price True
province True
region_1 True
region_2 True
taster_name True
taster_twitter_handle True
title False
variety True
winery False


In [38]:
# fix countries in df1 and df2, fix variety in df2
countries = {'Tsililis':'Greece', 'Büyülübağ':'Turkey', 'Chilcas':'Chile',
             'Gotsa Family Wines':'Georgia', 'Barton & Guestier':'France',
             'Kakhetia Traditional Winemaking':'Georgia', 'Ross-idi':'Bulgaria',
             'Orbelus':'Bulgaria', 'St. Donat':'Hungary', 'Familia Deicas':'Uruguay','Bartho Eksteen':'South Africa',
             'Stone Castle':'Kosovo', 'Teliani Valley':'Georgia', 'Undurraga':'Chile', 'Mt. Beautiful':'New Zealand',
             'Neumeister':'Austria', 'Santa Ema':'Chile', 'Psagot':'Israel',
             'Newton Johnson':'South Africa', 'Ktima Voyatzi':'Greece', 'Lismore':'South Africa', 'Logodaj':'Bulgaria',
             'Somlói Vándor':'Hungary', 'Les Frères Dutruy':'Switzerland', 'El Capricho':'Uruguay',
             'Amiran Vepkhvadze':'Georgia'}
#'Bachelder'
df2.at[38898, 'country'] = 'Canada'
df2.at[38898, 'region_1'] = 'Twenty Mile Bench'
df2.at[38898, 'region_2'] = 'Niagara Peninsula'
df2.at[86909, 'variety'] = 'Petite Syrah'
df2.at[86909, 'region_1'] = 'Maipo Valley'

for index, row in df1.iterrows():
    if df1.at[index, 'winery'] in countries:
      df1.at[index, 'country'] = countries[df1.at[index, 'winery']]
for index, row in df2.iterrows():
    if df2.at[index, 'winery'] in countries:
      df2.at[index, 'country'] = countries[df2.at[index, 'winery']]



print(df1[df1['country'].isnull()])
print(df2[df2['country'].isnull()])
print(df2[df2['variety'].isnull()])

Empty DataFrame
Columns: [id, country, description, designation, points, price, province, region_1, region_2, variety, winery]
Index: []
Empty DataFrame
Columns: [id, country, description, designation, points, price, province, region_1, region_2, taster_name, taster_twitter_handle, title, variety, winery]
Index: []
Empty DataFrame
Columns: [id, country, description, designation, points, price, province, region_1, region_2, taster_name, taster_twitter_handle, title, variety, winery]
Index: []


In [29]:
# fix designation in df1 and df2 // not fixable?
#print(df1[df1['designation'].isnull()])
#print(df2[df2['designation'].isnull()])

In [39]:
# fix regions in df1 and df2
region = np.concatenate((df1[df1['region_1'].isnull()].winery.unique(), df2[df2['region_1'].isnull()].winery.unique()))
region = np.concatenate((region, df1[df1['region_2'].isnull()].winery.unique()))
region = set(np.concatenate((region, df2[df2['region_2'].isnull()].winery.unique())))
print(len(region))
#df1.loc[(df1.region_2.isnull()) & (df1.region_1.isnull())])

14017


In [44]:
# merge regions in df1 and df2
# reviews.loc[(reviews.country == 'Italy') & (reviews.points >= 90)] // df2.at[index, 'winery']
#df1.loc[(df1.region_1 == df1.region_2) & (df1.region_1.notnull())]

for i, row in df1.iterrows():
  if df1.at[i, 'region_1'] != df1.at[i, 'region_2']:
    if len(df1.at[i, 'region_1']) == 0:
      df1.at[i, 'region_1'] = str(df1.at[i, 'region_2'])
    elif len(df1.at[i, 'region_2']) > 0:
      df1.at[i, 'region_1'] = (str(df1.at[i, 'region_1']) +", "+ str(df1.at[i, 'region_2']))

for i, row in df2.iterrows():
  if df2.at[i, 'region_1'] != df2.at[i, 'region_2']:
    if len(df2.at[i, 'region_1']) == 0:
      df2.at[i, 'region_1'] = str(df2.at[i, 'region_2'])
    elif len(df2.at[i, 'region_2']) > 0:
      df2.at[i, 'region_1'] = (str(df2.at[i, 'region_1']) +", "+ str(df2.at[i, 'region_2']))

df2.head()

AttributeError: 'str' object has no attribute 'notnull'

In [41]:
df1.head()

,id,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,"Napa Valley, Napa",Napa,Cabernet Sauvignon,Heitz
1,1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,"Toro, nan",NaN,Tinta de Toro,Bodega Carmen Rodríguez
2,2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,"Knights Valley, Sonoma",Sonoma,Sauvignon Blanc,Macauley
3,3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
4,4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,"Bandol, nan",NaN,Provence red blend,Domaine de la Bégude
